In [1]:
import tensorflow as tf
from tensorflow import keras

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import cv2
from imutils import paths
import shutil
import os
import csv
import glob
import gc
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [2]:
def labels2cat(label_encoder, list):
    return label_encoder.transform(list)

def load_csv_file():
    data = []
    for csv_file in tqdm(LESCO_DATA.path.values, total=len(LESCO_DATA.path.values)):
        with open(csv_file,'r') as f:
            lines = f.readlines()
            data.append(np.array([line.strip().split(',') for line in lines]))
    return np.vstack(data)

def load_csv_file_face():
    data = []
    for csv_file in tqdm(LESCO_DATA_FACE.path.values, total=len(LESCO_DATA_FACE.path.values)):
        with open(csv_file,'r') as f:
            lines = f.readlines()
            processed_lines = []
            for line in lines:
                columns = line.strip().split(',')
                processed_line = columns[:MAX_KEYPOINTS_FACE+3]  # Keep only the first MAX_KEYPOINTS_FACE+3 columns
                processed_lines.append(processed_line)
            data.append(np.array(processed_lines))
            if(len(lines)<MAX_FRAMES):
                print(csv_file)
            #if(len(np.array([line.strip().split(',') for line in lines])[1])<139):
                #print(csv_file)
    return np.vstack(data)

In [3]:
MAX_FRAMES = 30
MAX_KEYPOINTS = 134
MAX_KEYPOINTS_FACE=136 

ROOT_PATH=f"C:/Users/jimmy/Documents/ORIGINALES_RAW_{MAX_FRAMES}/"

In [4]:
## Get  

In [5]:
vid = {os.path.splitext(os.path.basename(x))[0]: x for x in glob.glob(ROOT_PATH + '*/*HP*.csv')}
LESCO_DATA = pd.DataFrame.from_dict(vid, orient = 'index').reset_index()
LESCO_DATA.columns = ['Class','path']
signers = LESCO_DATA.Class.str.split('_').str[0]
classes = LESCO_DATA.Class.str.split('_').str[1]
LESCO_DATA['Signer'] = signers
LESCO_DATA['Class'] = classes
vid_face = {os.path.splitext(os.path.basename(x))[0]: x for x in glob.glob(ROOT_PATH + '*/*BASE*_F_*.csv')}
LESCO_DATA_FACE = pd.DataFrame.from_dict(vid_face, orient = 'index').reset_index()
LESCO_DATA_FACE.columns = ['Class','path']


In [6]:
LESCO_DATA_FACE

,Class,path
0,Signer10_Ahorro_10_BASE_5N_F_0,C:/Users/jimmy/Documents/ORIGINALES_RAW_30\Aho...
1,Signer10_Ahorro_10_BASE_5N_F_1,C:/Users/jimmy/Documents/ORIGINALES_RAW_30\Aho...
2,Signer10_Ahorro_10_BASE_5N_F_2,C:/Users/jimmy/Documents/ORIGINALES_RAW_30\Aho...
3,Signer10_Ahorro_10_BASE_5_F_0,C:/Users/jimmy/Documents/ORIGINALES_RAW_30\Aho...
4,Signer10_Ahorro_10_BASE_5_F_1,C:/Users/jimmy/Documents/ORIGINALES_RAW_30\Aho...
...,...,...
33475,Signer9_Transferencia_9_BASE_FLIP_F_1,C:/Users/jimmy/Documents/ORIGINALES_RAW_30\Tra...
33476,Signer9_Transferencia_9_BASE_FLIP_F_2,C:/Users/jimmy/Documents/ORIGINALES_RAW_30\Tra...
33477,Signer9_Transferencia_9_BASE_F_0,C:/Users/jimmy/Documents/ORIGINALES_RAW_30\Tra...
33478,Signer9_Transferencia_9_BASE_F_1,C:/Users/jimmy/Documents/ORIGINALES_RAW_30\Tra...


In [7]:
all_classes = os.listdir(ROOT_PATH)
np.save("NPY_SPLITS/ALL_CLASSES.npy", all_classes)
le = LabelEncoder()
le.fit(all_classes)
list(le.classes_)
y_list = labels2cat(le, classes)
all_kps = load_csv_file()
all_kps = all_kps.reshape(-1, MAX_FRAMES, MAX_KEYPOINTS+3)
all_kps = all_kps[..., -MAX_KEYPOINTS:]
all_kps = all_kps.astype(np.float64)

100%|██████████| 33480/33480 [01:02<00:00, 533.81it/s]


In [8]:
all_kps.shape

(33480, 30, 134)

In [9]:
all_kps_face = load_csv_file_face()
all_kps_face = all_kps_face.reshape(-1, MAX_FRAMES, MAX_KEYPOINTS_FACE+3)
all_kps_face = all_kps_face[..., -MAX_KEYPOINTS_FACE:]
all_kps_face = all_kps_face.astype(np.float64)

100%|██████████| 33480/33480 [01:14<00:00, 448.30it/s]


In [10]:
all_kps_face.shape

(33480, 30, 136)

In [11]:
all_kps_final = np.concatenate((all_kps, all_kps_face), axis=2)
#all_kps_final = all_kps
all_kps_final.shape

(33480, 30, 270)

In [13]:
##Signer6 should not be used for validation - too little videos and some missing classes
for i in range(0, 5):
    if(i==0):
        match_strings = ['Signer1', 'Signer10']
    elif(i==1):
        match_strings = ['Signer2', 'Signer9']
    elif(i==2):
        match_strings = ['Signer3', 'Signer8']
    elif(i==3):
        match_strings = ['Signer4', 'Signer7']
    elif(i==4):
        match_strings = ['Signer5', 'Signer1']
    index = np.isin(signers.values, match_strings)
    train_list_HPF = all_kps_final[~index]
    train_label_HPF = y_list[~index]
    test_list_HPF = all_kps_final[index]
    test_label_HPF = y_list[index]
    train_filename = f"NPY_SPLITS/train_list_HPF_HOL_{MAX_FRAMES}_{i}.npy"
    np.save(train_filename, train_list_HPF)
    train_label_filename = f"NPY_SPLITS/train_label_HPF_HOL_{MAX_FRAMES}_{i}.npy"
    np.save(train_label_filename, train_label_HPF)
    test_filename = f"NPY_SPLITS/test_list_HPF_HOL_{MAX_FRAMES}_{i}.npy"
    np.save(test_filename, test_list_HPF)
    test_label_filename = f"NPY_SPLITS/test_label_HPF_HOL_{MAX_FRAMES}_{i}.npy"
    np.save(test_label_filename, test_label_HPF)
    
